#Modelling

Prepare dataset from others sources than audio

In [ ]:
import pandas as pd
import numpy as np
import os
import glob

columns = ['patient_id','audio_id', 'diagnose', 'audio_file', 'spectrogram_file','samsung','iphone','split_number','split_qty','less_A1097']
df_ = pd.DataFrame(columns=columns)

spectrograms_folder = 'audio_preparation/spectrograms'
diagnose_dir = 'diagnose/'
audios_features_folder = 'audios_features/'

list_subdirs = os.listdir(spectrograms_folder)
s=0
for i in list_subdirs:
    directory = spectrograms_folder+'/'+i+'/'
    list_jpg = glob.glob(directory+"*.png")
    if list_jpg:
        for k in range(len(list_jpg)):
            s+=1
            spectrogram_file = list_jpg[k].split('/')[-1]
            patient_id = spectrogram_file.split('_')[0]
            if int(patient_id.split('A')[-1])<1098:
                less_A1097 = 1
            else:
                less_A1097 = 0
            #
            if len(spectrogram_file.split('.')[0].split('_')[-1].split('s'))>1:
                samsung = 1
                iphone = 0
                audio_file = spectrogram_file.split('.')[0].split('_')[1]+'_'+spectrogram_file.split('.')[0].split('_')[2]+'.wav'
                audio_id = 'audio'+'_'+spectrogram_file.split('.')[0].split('_')[2]
                split_number = spectrogram_file.split('.')[0].split('_')[1].split('a')[0]
            elif len(spectrogram_file.split('.')[0].split('_')[-1].split('i'))>1:
                samsung = 0
                iphone = 1
                audio_file = spectrogram_file.split('.')[0].split('_')[1]+'_'+spectrogram_file.split('.')[0].split('_')[2]+'.wav'
                audio_id = 'audio'+'_'+spectrogram_file.split('.')[0].split('_')[2]
                split_number = spectrogram_file.split('.')[0].split('_')[1].split('a')[0]
            else:
                samsung = 0
                iphone = 0
                audio_file = spectrogram_file.split('.')[0].split('_')[1]+'_'+spectrogram_file.split('.')[0].split('_')[2]+'.wav'
                audio_id = 'audio'+'_'+spectrogram_file.split('.')[0].split('_')[2]
                split_number = spectrogram_file.split('.')[0].split('_')[1].split('a')[0]
            #
            df_.loc[s] = np.array([patient_id,audio_id, np.NaN, audio_file, spectrogram_file,samsung,iphone,split_number,np.NaN,less_A1097])
            patient_id
            audio_id

for patient_id in list(set(df_.patient_id)):
    for audio_id in list(set(df_[(df_.patient_id==patient_id)].audio_id)):
        dim = df_[(df_.patient_id==patient_id)&(df_.audio_id==audio_id)].shape[0]
        df_.loc[(df_.patient_id==patient_id)&(df_.audio_id==audio_id), ['split_qty']] = dim


#df_=pd.load(pandas_dir+pandas_file)


diagnose_file = 'Diagnozes.csv'
diagnose = pd.read_csv(diagnose_dir+diagnose_file,sep=';')



# Assign spreadsheet filename to `file`
file = diagnose_dir+'Diagnozes_2016.03.01.xlsx'
#file = diagnose_dir+'Mano duomenys.xls'


xl = pd.ExcelFile(file)
#diagnose = xl.parse('Data')

# Print the sheet names
#print(xl.sheet_names)

diagnose = xl.parse('Sheet1')

diagnose = diagnose.rename(columns = {'File':'patient_id'})
diagnose = diagnose.rename(columns = {'Dgn.':'sub_diagnose'})


df_1 = pd.merge(df_, diagnose, how='left', on=['patient_id'])
df_1.loc[(df_1.sub_diagnose==3), ['diagnose']] = 1
df_1.loc[(df_1.sub_diagnose!=3), ['diagnose']] = 0

df_ = df_1

if not os.path.exists(audios_features_folder):
    os.makedirs(audios_features_folder)

df_.to_pickle(audios_features_folder+'target_data.p') 
#df_ = pd.read_pickle(audios_features_folder+'target_data')

Setup modelling environment 

In [ ]:
#Required parameters for modelling
audio_record_devices_for_training = ['saumsung','iphone','studio']

audio_record_devices_for_validating = ['iphone']

audio_features = ['spectrogram','power_spectrum','soundnet','patient_survey']

features_reduction_algorithm = ['PCA']

features_after_reduction_size = 30

prediction_algorithms = ['SVM','RandomForest','ELM','KNN']

target = ['sick_healthy','all_diseases']

sick_percentage_in_sample = 0.5

sample_for_training_percentage = 0.6

max_records_from_one_track = 20

max_tracks_from_one_person = 20



#Optional parameters

soundnet_layers = [23,25]

In [ ]:
#Filter by device records
target_data = pd.DataFrame()

if ('samsung' in audio_record_devices):
    target_data = target_data.append(df_[df_.samsung == '1'], ignore_index = True)

if ('iphone' in audio_record_devices):
    target_data = target_data.append(df_[df_.iphone == '1'], ignore_index = True)

if ('studio' in audio_record_devices):
    target_data = target_data.append(df_[(df_.samsung == '0') & (df_.iphone == '0')], ignore_index = True)


#Split to training, testing and validation datasets
from sklearn.model_selection import train_test_split

patient_id_train, patient_id_test_temp = train_test_split(target_data.patient_id.unique().tolist(), train_size = sample_for_training_percentage)
patient_id_test, patient_id_validate = train_test_split(patient_id_test_temp, train_size = 0.5)


target_data.loc[df_.patient_id.isin(patient_id_train),'patient_split'] = 'training'
target_data.loc[df_.patient_id.isin(patient_id_test),'patient_split'] = 'testing'
target_data.loc[df_.patient_id.isin(patient_id_validate),'patient_split'] = 'validating'

target_data[(target_data.patient_split=='validating')&(target_data.iphone=='1')].groupby(['patient_id']).max().groupby(['diagnose']).count()


In [ ]:
import random
import pickle


audios_features_folder = 'audios_features'
#import features extracted from spectrograms
features_from_spectrograms = pickle.load( open( audios_features_folder+"/"+"audios_features.p", "rb" ) )


#import features extracted from spectrograms
features_from_power_spectrum = pickle.load( open( audios_features_folder+"/"+"audios_features_power_spectrum.p", "rb" ) )


#Get data for training
patient_id_train = target_data[target_data.patient_split=='training'].patient_id.unique().tolist()

target_data[target_data.patient_id=='A1475'].audio_id.unique()


train_x_dataset = []
train_y_dataset = []
for patien_id in patient_id_train:
    audio_tracks = target_data[target_data.patient_id==patien_id].audio_id.unique()
    selected_audio_tracks = random.sample(audio_tracks, min(audio_tracks.__len__(),max_tracks_from_one_person))
    
    for audio_track in selected_audio_tracks:
        spectrogram_files = target_data[(target_data.patient_id==patien_id)&(target_data.audio_id==audio_track)].spectrogram_file.unique()
        selected_spectrograms_from_track = random.sample(spectrogram_files, min(spectrogram_files.__len__(),max_records_from_one_track))
        
        if 'spectrogram' in audio_features:
            
        
        if 'power_spectrum' in audio_features:
            
        
        if 'soundnet' in audio_features:
            
            
        if 'patient_survey' in audio_features:
            
        
        
        
        train_x.append(img)
        

#Get data for testing
test_x_dataset = []
test_y_dataset = []

patient_id_test = target_data[target_data.patient_split=='testing'].patient_id.unique().tolist()

#Get data for validating
validate_x_dataset = []
validate_y_dataset = []

patient_id_validate = target_data[target_data.patient_split=='validating'].patient_id.unique().tolist()